In [56]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'

In [39]:
def load(paths, verbose=0):
  data = list()
  labels = list()

  for (i, imgpath)in enumerate(paths):
    im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
    image = np.array(im_gray).flatten()
    
    #label of the image is extracted from the image path (selecting the second to last element of the resulting list, which is assumed to be the label of the image)
    label = imgpath.split(os.path.sep)[-2]
    data.append(image/255)
    labels.append(label)

    if verbose >0 and i>0 and (i+1) %verbose ==0 :
      print("[INFO] processed {}/{}".format(i+1,len(paths)))
  return data,labels

In [48]:
img_path ='./archive/trainingSet'
from imutils import paths
image_paths = list(paths.list_images(img_path))
image_list, label_list = load(image_paths,verbose=10000)

lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

#split data fro trainig and testing
X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.1, random_state=42)

[INFO] processed 10000/42000
[INFO] processed 20000/42000
[INFO] processed 30000/42000
[INFO] processed 40000/42000


In [49]:
def create_clients(image_list, label_list, num_clients=100,initial='clients'):

  #create a list of client names
  client_names =['{}_{}'.format(initial, i+1) for i in range(num_clients)]

  #randomize data
  data = list(zip(image_list, label_list))
  random.shuffle(data)
  
  #sharing data with particular each client
  size = len(data)//num_clients
  shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

  #code asserts that the number of clients is equal to the number of shards, 
  assert(len(shards) == len(client_names))
 
  #returns  [key]= client name and [value] = corresponding shard of data for that client.
  return {client_names[i] : shards[i] for i in range(len(client_names))}

In [50]:
client = create_clients(X_train, y_train, num_clients=100, initial='clients')

In [51]:
def batch_data(data_shard, batch_size =32):
   
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(batch_size)

In [55]:
#batch the training data for each clients
clients_batched = dict()

for(client_names, data) in client.items():
      clients_batched[client_names] = batch_data(data)

#batch the test data
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

NameError: name 'tf' is not defined